## Text recognition with Computer Vision API  <a name="RecognizeText"> </a>
* [Detect and extract printed text from an image](#OCR)
* [Detect and extract handwritten text from an image](#RecognizeText)

Use the [Recognize Text method](https://westcentralus.dev.cognitive.microsoft.com/docs/services/5adf991815e1060e6355ad44/operations/587f2c6a154055056008f200) to asynchronously detect handwritten or printed text in an image and extract recognized characters into a machine-usable character stream.

Set `image_path` to point to the image to be recognized.

In [ ]:
# IMPORTING THE LIBRARIES
import requests
%matplotlib inline
from PIL import Image
import matplotlib.patches as patches
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
import time
from matplotlib.patches import Polygon

# PROXIES IF YOU ARE RUNNING THE JUPYTER NOTEBOOK FROM YOUR LAPTOP
#proxy = "http://www.com:8080"
#proxys = "https://www.com:8080"
proxyDict = { "http"  : proxy, "https"  : proxys }

In [ ]:
subscription_key = 'put_your_key_here'
assert subscription_key

image_path = "FC_OCR2.PNG" # CHOOSE YOUR LOCAL IMAGE FOR RECOGNITION HERE

In [ ]:
vision_base_url = "https://PYTHONCOMPUTERVISION.cognitiveservices.azure.com/vision/v1.0/"
ocr_url = vision_base_url + "recognizeText"
print(ocr_url)

In [ ]:

image_data = open(image_path, "rb").read()
headers    = {'Ocp-Apim-Subscription-Key': subscription_key, 
              "Content-Type": "application/octet-stream" }

params   = {'mode' : 'Handwritten'}
#data     = {'url': image_url}
response = requests.post(ocr_url, headers=headers, params=params, data=image_data) #, proxies=proxyDict)
response.raise_for_status()

operation_url = response.headers["Operation-Location"]

analysis = {}
while not "recognitionResult" in analysis:
    response_final = requests.get(response.headers["Operation-Location"], headers=headers, proxies=proxyDict)
    analysis       = response_final.json()
    time.sleep(1)

polygons = [(line["boundingBox"], line["text"]) for line in analysis["recognitionResult"]["lines"]]


Next, the recognized text along with the bounding boxes can be extracted as shown in the following line of code. An important point to note is that the handwritten text recognition API returns bounding boxes as **polygons** instead of **rectangles**. Each polygon is _p_ is defined by its vertices specified using the following convention:

<i>p</i> = [<i>x</i><sub>1</sub>, <i>y</i><sub>1</sub>, <i>x</i><sub>2</sub>, <i>y</i><sub>2</sub>, ..., <i>x</i><sub>N</sub>, <i>y</i><sub>N</sub>]

In [ ]:
# PLOTTING RECTANGLES ON RECOGNIZED TEXT
plt.figure(figsize=(25,25))
#image  = Image.open(BytesIO(requests.get(image_url).content))
image = Image.open(image_path)
ax     = plt.imshow(image)
for polygon in polygons:
    vertices = [(polygon[0][i], polygon[0][i+1]) for i in range(0,len(polygon[0]),2)]
    text     = polygon[1]
    print(text,vertices)
    patch    = Polygon(vertices, closed=True,fill=False, linewidth=2, color='y')
    ax.axes.add_patch(patch)
    plt.text(vertices[0][0], vertices[0][1], text, fontsize=15, va="top")
_ = plt.axis("off")


In [ ]:
for polygon in polygons:
    vertices = [(polygon[0][i], polygon[0][i+1]) for i in range(0,len(polygon[0]),2)]
    text     = polygon[1]
    print (text,vertices)